<a href="https://colab.research.google.com/github/Niknik26/14_final_task/blob/main/14_11_Final_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, mean, max, avg, month, year, count, sum

# Создание SparkSession
spark = SparkSession.builder.appName("Read weather").getOrCreate()

# Чтение CSV-файла
df = spark.read.csv("/content/web_server_logs.csv", header=True, inferSchema=True)
#df.show()

# 1. Сгруппируйте данные по IP и посчитайте количество запросов для каждого IP, выводим 10 самых активных IP.
grouped_ip = df.groupBy("ip").agg(count(col("response_code")).alias("request_count")).orderBy(col("request_count").desc()).limit(10)
print("Top 10 active IP addresses:")
grouped_ip.show()

# 2. Сгруппируйте данные по HTTP-методу и посчитайте количество запросов для каждого метода
grouped_method = df.groupBy("method").agg(count(col("response_code")).alias("method_count"))
print("Request count by HTTP method:")
grouped_method.show()

# 3. Профильтруйте и посчитайте количество запросов с кодом ответа 404.
count404 = df.filter(df.response_code == 404).count()
print(f"Number of 404 response codes: {count404}")

# 4. Сгруппируйте данные по дате и просуммируйте размер ответов, сортируйте по дате.

# Преобразование столбца timestamp в формат даты
date_with = df.withColumn("date", to_date(col("timestamp"), "yyyy-MM-dd"))

# Суммируем по дате
filter404 = date_with.groupBy("date").agg(sum(col("response_size")).alias("total_response_size")).orderBy(col("date").asc())
print("Total response size by day:")
filter404.show()


Top 10 active IP addresses:
+--------------+-------------+
|            ip|request_count|
+--------------+-------------+
|  12.202.59.29|            2|
|   57.60.34.63|            2|
| 178.175.35.67|            2|
|   200.226.2.8|            1|
|77.109.193.133|            1|
| 101.213.67.53|            1|
|141.102.15.115|            1|
|80.211.210.198|            1|
|107.91.132.207|            1|
|  105.213.0.28|            1|
+--------------+-------------+

Request count by HTTP method:
+------+------------+
|method|method_count|
+------+------------+
|  POST|       25221|
|DELETE|       24787|
|   PUT|       24945|
|   GET|       25047|
+------+------------+

Number of 404 response codes: 25142
Total response size by day:
+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2024-01-01|            2088095|
|2024-01-02|            2134155|
|2024-01-03|            2401391|
|2024-01-04|            2403109|
|2024-01-05|            2183526|
|2